# Исследование надёжности заёмщиков

### Оглавление
1. [Открытие файла с данными и изучение общей информации](#чтение_файла)
2. [Предобработка данных](#предобработка_данных)
    * [Обработка пропусков](#предобработка_данных__обработка_пропусков)
    * [Замена типа данных](#замена_типа_данных)
    * [Обработка дубликатов](#обработка_дубликатов)
    * [Лемматизация](#лемматизация)
    * [Категоризация данных](#категоризация_данных)
3. [Ответы на вопросы](#ответы_на_вопросы)
4. [Общий вывод](#общий_вывод)

### Шаг 1. Открытие файла с данными и изучение общей информации.<a id="чтение_файла"></a>

Импортируем библиотеку Pandas, прочитаем таблицу с данными, используя метод `read_csv()` и посмотрим обущую информацию о таблице.

In [ ]:
import pandas as pd
data = pd.read_csv()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Выведем первые 10 строк таблицы, используя метод `head(10)`.

In [ ]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [ ]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

- Таблица содержит 21525 строк и 12 столбцов;

- 2 столбца имеют  тип данных float64, 5 столбцов int64 и 5 столбцов object;

- Столбцы days_employed и total_income содержат пропущенные значения, которые необходимо дополнительно исследовать. Количество пропусков в этих столбцах одинаково;

- Столбец days_employed содержит отрицательные данные, так и выбросы (например строка с индексом 4);

- Столбец education содержит значения с разным регистром;

- Столбец gender содержит значения, которые написаны на английском языке, хотя все остальные столбцы написаны на русском языке;

- Столбец purpose содержит в себе очень много уникальных значений, но смысл многих пересекается;

- Названия столбцов можно трактовать по-разному, поэтому необходимо будет их переименовать.

### Шаг 2. Предобработка данных<a id="предобработка_данных"></a>

### 2.1 Обработка пропусков<a id="предобработка_данных__обработка_пропусков"></a>

Проверим каждый столбец на наличие аномальных данных и выбросов

#### 2.1.1 Столбец *children*<a id="предобработка_данных__обработка_пропусков__столбец_children"></a>

In [ ]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце есть два аномальных значения: -1, 20. Посмотрим целиком строки, содержащие эти значения.

In [ ]:
data[data['children'] == -1].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля


In [ ]:
data[data['children'] == 20].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля


Как видим эти строки ничем не выделяются по содержимому, поэтому предположим, что при заполнении данных были допущены опечатки. Был случайно добавлен знак минус для значения -1 и лишний ноль для значения 20. Поэтому заменим -1 на 1, а 20 на 2 с помощью метода `replace()` и проверим, что замена произошла.

In [ ]:
data['children'] = data['children'].replace(-1,1)
data['children'] = data['children'].replace(20,2)
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

В результате видим, что теперь столбец ***children*** не содержит аномальных значений.

#### 2.1.2 Столбец *dob_years*<a id="предобработка_данных__обработка_пропусков__столбец_dob_years"></a>

Теперь исследуем столбец ***dob_years***. Посмотрим минимальный и максимальный возраст клиентов.

In [ ]:
dob_years = data['dob_years'].min()
print('Минимальный возраст клиента:', dob_years)

Минимальный возраст клиента: 0


In [ ]:
dob_years = data['dob_years'].max()
print('Максимальный возраст клиента:', dob_years)

Максимальный возраст клиента: 75


Из результатов видно, что в столбце есть пропуски, так как возраст не может быть 0. Посмотрим целиком строки, содержащие эти значения и их количество.

In [ ]:
data[data['dob_years'] == 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


In [ ]:
print('Количество пропусков в столбце dob_years:', data[data['dob_years'] == 0]['children'].count())

Количество пропусков в столбце dob_years: 101


У 101 клиента отсутствует возраст. Исправим эту ситуацию. Посмотрим средний возраст клиентов в зависимости от их занятости без учета клиентов с возрастом 0.

In [ ]:
dob_years_mean = data[data['dob_years'] !=0 ].groupby('income_type')['dob_years'].mean()
dob_years_mean

income_type
безработный        38.000000
в декрете          39.000000
госслужащий        40.804542
компаньон          39.854294
пенсионер          59.370959
предприниматель    42.500000
сотрудник          40.018980
студент            22.000000
Name: dob_years, dtype: float64

Напишем функцию, которая будет заменять в столбце ***dob_years*** значения 0 на средний возраст в зависимости от типа занятости и применим ее к нашей таблице.

In [ ]:
def dob_years_no_empty(row):
    """"
    функция определяет, если возраст клиента <=0 или >100
    и заменяет его на средний возраст, округлив до ближайшего целого,
    в зависимости от типа занятости
    """
    dob_years   = row['dob_years']
    income_type = row['income_type']

    if (dob_years <= 0) | (dob_years > 100):
        return round(dob_years_mean[income_type])
    else:
        return dob_years

data['dob_years'] = data.apply(dob_years_no_empty, axis=1)

Проверим еще раз минимальный и максимальный возраст клиентов.

In [ ]:
dob_years = data['dob_years'].min()
print('Минимальный возраст клиента:', dob_years)

Минимальный возраст клиента: 19


In [ ]:
dob_years = data['dob_years'].max()
print('Максимальный возраст клиента:', dob_years)

Максимальный возраст клиента: 75


В результате мы избавились от пустых значений в столбце ***dob_years***.

#### 2.1.3 Столбец *education*<a id="предобработка_данных__обработка_пропусков__столбец_education"></a>

Теперь исследуем столбец ***education*** на пустые значения и аномальные данные.

In [ ]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Пустых и аномальных данных нет, но необходимо все значения привести к нижнему регистру.

In [ ]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

#### 2.1.4 Столбец *education_id*<a id="предобработка_данных__обработка_пропусков__столбец_education_id"></a>

Проверим столбец ***education_id*** на пустые значения и аномальные данные.

In [ ]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

В этом столбце все данные адекватные.

#### 2.1.5 Столбец *family_status*<a id="предобработка_данных__обработка_пропусков__столбец_family_status"></a>

Проверим столбец ***family_status*** на пустые значения и аномальные данные.

In [ ]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Пустых и аномальных данных нет, но необходимо все значения привести к нижнему регистру.

In [ ]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

#### 2.1.6 Столбец *family_status_id*<a id="предобработка_данных__обработка_пропусков__столбец_family_status_id"></a>

Проверим столбец ***family_status_id*** на пустые значения и аномальные данные.

In [ ]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

#### 2.1.7 Столбец *gender*<a id="предобработка_данных__обработка_пропусков__столбец_gender"></a>

Проверим столбец ***gender*** на пустые значения и аномальные данные.

In [ ]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

XNA? Найдем строчку с этим значением.

In [ ]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Так как строчка содержащая XNA всего одна и пол определить не получится, то удалим эту сточку.

In [ ]:
data = data[data['gender'] != 'XNA']

Проверим, что строчки с XNA больше нет.

In [ ]:
data[data['gender'] == 'XNA']['debt'].count()

0

#### 2.1.8 Столбец *income_type*<a id="предобработка_данных__обработка_пропусков__столбец_income_type"></a>

Проверим столбец ***income_type*** на пустые значения и аномальные данные.

In [ ]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

В этом столбце все данные адекватные.

#### 2.1.9 Столбец *debt*<a id="предобработка_данных__обработка_пропусков__столбец_debt"></a>

Проверим столбец ***debt*** на пустые значения и аномальные данные.

In [ ]:
data['debt'].value_counts()

0    19783
1     1741
Name: debt, dtype: int64

В этом столбце все данные адекватные.

#### 2.1.10 Столбец *purpose*<a id="предобработка_данных__обработка_пропусков__столбец_purpose"></a>

Проверим столбец ***purpose*** на пустые значения и аномальные данные.

In [ ]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Пропущенных и аномальных значений в этом столбце нет.

#### 2.1.11 Столбец *total_income*<a id="предобработка_данных__обработка_пропусков__столбец_total_income"></a>

Как мы выяснили раннее, столбцы ***total_income*** и ***days_employed*** имеют пропущенные значения. Их количество одинаково. Предположим, что значения пропущены одновременно в двух столбцах. Проверим эту гипотезу.

Для начала найдем количество пропусков в столбце ***total_income***.

In [ ]:
print('Количесто пропусков в столбце total_income:', data[data['total_income'].isnull()]['income_type'].count())

Количесто пропусков в столбце total_income: 2174


И количество пропусков в столбце ***days_employed***.

In [ ]:
print('Количесто пропусков в столбце days_employed:', data[data['days_employed'].isnull()]['income_type'].count())

Количесто пропусков в столбце days_employed: 2174


Теперь проверим нашу гипотезу...

In [ ]:
data[(data['total_income'].isnull()) & (data['days_employed'].isnull())]['income_type'].count()

2174

Действительно, если значение пропущено в столбце ***total_income***, то оно и пропущено в столбце ***days_employed***.

 Попробуем их восстановить. Начнем со столбца ***total_income***.

Посмотрим минимальное и максимальное значения уровня дохода клиентов.

In [ ]:
total_income_min = data['total_income'].min()
print('Минимальный уровень дохода клиентов: {:.2f}'.format(total_income_min))

Минимальный уровень дохода клиентов: 20667.26


In [ ]:
total_income_max = data['total_income'].max()
print('Минимальный уровень дохода клиентов: {:.2f}'.format(total_income_max))

Минимальный уровень дохода клиентов: 2265604.03


Заменим NaN на 0 для столбца ***total_income***.

In [ ]:
data['total_income'] = data['total_income'].fillna(0) 

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       19350 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Отрицательных и резко выделяющихся значений нет. Осталось избавиться от пропусков. Посмотрим медиану уровня дохода в зависимости от типа занятости клиента, но без учета клиентов, у которых был пропущен уровень дохода.

In [ ]:
total_income_median = data[data['total_income'] > 0 ].groupby('income_type')['total_income'].median()
total_income_median

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172319.266339
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Посчитаем количсетво пропущенных значений.

In [ ]:
print('Количество пропущенных значений:', data[data['total_income'] == 0]['income_type'].count())

Количество пропущенных значений: 2174


Напишем функцию, которая будет заменять в столбце ***dob_years*** значения 0 на средний возраст в зависимости от типа занятости и применим ее к нашей таблице.

In [ ]:
def total_income_no_empty(row):
    """"
    функция определяет текущий уровень дохода клиента и если он пропущен, то
    заменяет его на медиану уровня дохода в зависимости от типа занятости
    """
    total_income = row['total_income']
    income_type  = row['income_type']
    
    if total_income == 0:
        return total_income_median[income_type]
    else:
        return total_income

data['total_income'] = data.apply(total_income_no_empty, axis=1)

Посчитаем количество пропущенных значений после выполнения функции.

In [ ]:
print('Количество пропущенных значений:', data[data['total_income'] == 0]['income_type'].count())

Количество пропущенных значений: 0


С пропущенным значениями в столбце ***total_income*** мы разобрались.

#### 2.1.12 Столбец *days_employed*<a id="предобработка_данных__обработка_пропусков__столбец_days_employed"></a>

Перейдем к столбцу ***days_employed***. В этом столбце есть пропущенные знаечния, отрицательные значения и выбросы. 

Для начала разберемся с выбросами в столбце. Отсортируем столбец days_employed для знаений больше 0, есть предположения, что все выбросы положительные.


In [ ]:
data[data['days_employed'] > 0]['days_employed'].sort_values()

20444    328728.720605
9328     328734.923996
17782    328771.341387
14783    328795.726728
7229     328827.345667
             ...      
7794     401663.850046
2156     401674.466633
7664     401675.093434
10006    401715.811749
6954     401755.400475
Name: days_employed, Length: 3445, dtype: float64

Действительно все выбросы положительные. Количество выбросов 3445. Выведем 10 строк, которые содержат выбросы.

In [ ]:
data[data['days_employed'] > 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


В глаза сразу бросается столбик ***income_type***. Все выбросы только у пенсионеров?. Проверим эту гипотезу.

In [ ]:
data[data['days_employed'] > 0]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Помимо пенсионеров выбросы оказались еще у 2 безработных. А сколько всего у нас безработных и пенсионеров?

In [ ]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Количество выбросов для безработных и общее число безработных клиентов совпадают. Но для пенсионеров эти знчения разные. Предположим, что у оставшихся пенсионеров, отсутствуют данные по трудовому стажу... Проверим эту гипотезу.

In [ ]:
data[((data['days_employed'] > 0) | (data['days_employed'].isnull())) & (data['income_type'] == 'пенсионер')]['income_type'].count()

3856

Гипотеза верна. Если клиент является пенсионером, то в столбце со стажем значение будет является выбросом или вообще отсутствовать.

Скорее всего для выбросов вместо дней здесь указаны часы. Данные взяты из другой таблицы или раньше стаж указывали в часах. Поэтому напишем функцию, которая заменяет часы на дни для выбросов.

In [ ]:
def hours_to_days(days_employed):
    """"
    функция определяет текущей значение стажа, если оно положительное, то
    конвертирует его из часов в дни
    """
    if days_employed > 0:
        return days_employed / 24
    else:
        return days_employed

data['days_employed'] = data['days_employed'].apply(hours_to_days)

Проверим еще раз, что данные заменились корректно. 

In [ ]:
data[data['days_employed'] > 0]['days_employed'].sort_values()

20444    13697.030025
9328     13697.288500
17782    13698.805891
14783    13699.821947
7229     13701.139403
             ...     
7794     16735.993752
2156     16736.436110
7664     16736.462226
10006    16738.158823
6954     16739.808353
Name: days_employed, Length: 3445, dtype: float64

Мы избавились от выбросов в столбце ***days_employed***.

Теперь возьмем по модулю все значения, хранящиеся в этом столбце. И проверим, что они теперь все больше нуля.

In [ ]:
data['days_employed'] = data['days_employed'].abs()
data[data['days_employed'] < 0]['total_income'].count()

0

Посмотрим медиану стажа в зависимости от возраста клиента.

In [ ]:
days_employed_mean = data.groupby('dob_years')['days_employed'].median()
days_employed_mean

dob_years
19      724.492610
20      674.838979
21      618.733817
22      703.310078
23      690.204208
24      942.390603
25      919.199388
26     1083.658132
27     1166.212160
28     1141.705450
29     1315.453550
30     1420.586863
31     1308.901135
32     1446.622542
33     1426.415003
34     1615.910188
35     1613.494818
36     1799.520465
37     1816.713382
38     1817.194286
39     1891.388566
40     1669.279681
41     1877.153797
42     2253.981630
43     1869.019899
44     2084.330015
45     2254.431130
46     2100.473217
47     2203.078545
48     2429.674518
49     2560.317106
50     2626.986257
51     2846.080700
52     3395.639568
53     3650.007523
54     4026.541145
55     7043.471139
56     6850.179463
57    13926.326157
58    13817.626857
59    14439.234121
60    14574.440068
61    14371.636007
62    14790.904924
63    14936.762331
64    14840.866936
65    14902.851210
66    15159.476623
67    14884.285634
68    15219.924774
69    15096.390637
70    15116.157187
71

Заменим NaN на 0 для столбца ***days_employed*** и проверим, что в столбце больше нет NaN значений.

In [ ]:
data['days_employed'] = data['days_employed'].fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Посчитаем количество пропущенных значений.

In [ ]:
print('Количество пропущенных значений:', data[data['days_employed'] == 0]['days_employed'].count())

Количество пропущенных значений: 2174


Напишем функцию, которая будет заменять в столбце ***days_employed*** значения 0 на медиану значения стажа клиента в зависимости от возраста и применим ее к нашей таблице.

In [ ]:
def days_employed_no_empty(row):
    """"
    функция определяет текущий уровень стажа клиента и если он пропущен, то
    заменяет его на медиану стажа в зависимости от типа занятости
    """
    dob_years     = row['dob_years']
    days_employed = row['days_employed']
    
    if days_employed == 0:
        return days_employed_mean[dob_years]
    else:
        return days_employed

data['days_employed'] = data.apply(days_employed_no_empty, axis=1)

Посчитаем количество пропущенных значений после выполнения функции.

In [ ]:
print('Количество пропущенных значений:', data[data['days_employed'] == 0]['days_employed'].count())

Количество пропущенных значений: 0


Посмотрим еще раз информацию о таблице.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

Проверили все столбцы на наличие пропущенных значений и выбросов. В результате нам удалось избавиться от всех пропущенных данных. Также удалось избавиться от выбросов.

### 2.2 Замена типа данных<a id="замена_типа_данных"></a>

Выведем еще раз данные о таблице.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Видно, что столбцы ***days_employed*** и ***total_income*** принадлежат к типу float. Изменим тип данных на int, чтобы стаж и зарплата были целочисленными данными.

In [ ]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Проверим, что тип данных изменился.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Заменили на целочисленный тип данных (int) столбцы ***days_employed*** и ***total_income***, так как стаж и зарплата не могут быть вещественными числами (float).

### 2.3 Обработка дубликатов<a id="обработка_дубликатов"></a>

Посмотрим количество дубликатов, содержащихся в таблице.

In [ ]:
print('Дубликатов в таблице:', data.duplicated().sum())

Дубликатов в таблице: 71


Удалим все дубликаты и восстановим индексы.

In [ ]:
data = data.drop_duplicates().reset_index(drop = True)

Посчитаем еще раз количество дубликатов.

In [ ]:
print('Дубликатов в таблице после их удаления:', data.duplicated().sum())

Дубликатов в таблице после их удаления: 0


### Вывод

Удалось найди в таблице одинаковые строки, их получилось 71. Используя метод `drop_duplicates()` удалось избавиться от дубликатов. Возможная причина возникновения дубликатов - это человеческий фактор, когда сотрудник банка случайно отправляет несколько заявок на кредит. Еще возможная причина, что клиент сам несколько раз подавал заявку на кредит, но тогда значения трудового стажа должны были быть разными.

### 2.4 Лемматизация<a id="лемматизация"></a>

Посмотрим и посчитаем все цели кредита.

In [ ]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Напишем функцию, которая проведет лемматизацию цели кредита и добавит реузльтат в отдельный список для всех клиентов.

In [ ]:
from pymystem3 import Mystem
m = Mystem()
lemmas_result = []

def lemmas_all(row):
    """
    функция принимает строку, проводит лемматизацию для столбца purpose
    и добавляет результаты в список lemmas_result
    """
    purpose = row['purpose']
    lemmas = m.lemmatize(purpose)
    lemmas_result.extend(lemmas)

data.apply(lemmas_all, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
21448    None
21449    None
21450    None
21451    None
21452    None
Length: 21453, dtype: object

Посчитаем количество слов в списке `lemmas_result`.

In [ ]:
from collections import Counter
print(Counter(lemmas_result)) 

Counter({' ': 33569, '\n': 21453, 'недвижимость': 6350, 'покупка': 5896, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


### Вывод

Провели лемматизацию цели кредита. Выявили 5 основных целей: недвижимость, жилье, автомобиль, образование, свадьба.

### 2.5 Категоризация данных<a id="категоризация_данных"></a>

Напишем функцию, которая будет делить данные в столбце на 4 части. В качестве аргументов передаются `DataFrame` и название столбца. Функция возвращает 3 точки, которые соответствуют 25%, 50%, 75%.

In [ ]:
def quartile(data, columns_name):
    """
    Функция делит данные в столбце на 4 равные части. 
    В качестве аргументов передаются DataFrame и название столбца. 
    Функция возвращает 3 точки, которые соответствуют 25%, 50%, 75%.
    """
    level_50_percent = data[columns_name].median()
    level_25_percent = data[data[columns_name] < level_50_percent][columns_name].median()
    level_75_percent = data[data[columns_name] > level_50_percent][columns_name].median()
    
    return [level_25_percent, level_50_percent, level_75_percent]

#### 2.5.1 Категории для *dob_years*.<a id="категоризация_данных__категории_для_dob_years"></a>

In [ ]:
quartile(data, 'dob_years')

[33.0, 42.0, 53.0]

В результате получаем 4 категории: до 33 лет, от 33 до 42 лет, от 42 лет до 53 лет, более 53 лет.

Напишем функцию, которая принимает возраст клиента и причисляет его к определенной категории.

In [ ]:
def dob_years_group(age):
        """
        Возвращает возрастную группу по значению возраста age.
        Выделяют 4 категории:
        - до 33 лет;
        - от 33 до 42 лет;
        - от 42 лет до 53 лет;
        - более 53 лет.
        """

        if age < 33:
                return 'до 33 лет'
        elif 33 <= age < 42:
                return 'от 33 до 42 лет'
        elif 42 <= age < 53:
                return 'от 42 лет до 53 лет'
        return 'более 53 лет'

Cоздадим отдельный столбец с возрастными категориями, и в его ячейках запишем значения, возвращаемые функцией.

In [ ]:
data['dob_years_group'] = data['dob_years'].apply(dob_years_group)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 42 лет до 53 лет
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 33 до 42 лет
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 33 до 42 лет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 33 лет
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,более 53 лет
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,до 33 лет
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,от 42 лет до 53 лет
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,от 42 лет до 53 лет
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,от 33 до 42 лет
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,от 33 до 42 лет


Выведем статистику по возрастной группе методом `value_counts()`.

In [ ]:
data['dob_years_group'].value_counts()

более 53 лет           5734
от 42 лет до 53 лет    5585
от 33 до 42 лет        5350
до 33 лет              4784
Name: dob_years_group, dtype: int64

#### 2.5.2 Категории для *total_income*.<a id="категоризация_данных__категории_для_total_income"></a>

In [ ]:
quartile(data, 'total_income')

[103460.0, 142594.0, 196580.5]

В результате получаем 4 зарплатных категории: до 103460, от 103460 до 142594, от 142594 до 196593, более 196593.

Напишем функцию, которая принимает уровень дохода клиента и причисляет его к определенной категории.

In [ ]:
def total_income_group(total_income):
        """
        Возвращает группу уровня дохода по значению total_income.
        Выделяют 4 категории:
        - до 103460           - малоимущие;
        - от 103460 до 142594 - низкий доход;
        - от 142594 до 196593 - средний доход;
        - более 196593        - высокий доход.
        """
        if total_income < 103460:
                return 'малоимущие'
        elif 103460 <= total_income < 142594:
                return 'низкий доход'
        elif 142594 <= total_income < 196593:
                return 'средний доход'
        return 'высокий доход'

Cоздадим отдельный столбец с зарплатными категориями, и в его ячейках запишем значения, возвращаемые функцией.

In [ ]:
data['total_income_group'] = data['total_income'].apply(total_income_group)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 42 лет до 53 лет,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 33 до 42 лет,низкий доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 33 до 42 лет,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 33 лет,высокий доход
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,более 53 лет,средний доход
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,до 33 лет,высокий доход
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,от 42 лет до 53 лет,высокий доход
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,от 42 лет до 53 лет,низкий доход
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,от 33 до 42 лет,малоимущие
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,от 33 до 42 лет,средний доход


Выведем статистику по группе методом `value_counts()`.

In [ ]:
print(data['total_income_group'].value_counts()) 

средний доход    6375
высокий доход    5305
низкий доход     4887
малоимущие       4886
Name: total_income_group, dtype: int64


#### 2.5.3 Категории для *days_employed*.<a id="категоризация_данных__категории_для_days_employed"></a>

In [ ]:
quartile(data, 'days_employed')

[1004.0, 2167.0, 5472.0]

В результате получаем 4 категории стажа: до 1004 дней, от 1004 до 2167 дней, от 2167 до 5471 дней, более 5471 дней.

Напишем функцию, которая принимает стаж клиента и причисляет его к определенной категории.

In [ ]:
def days_employed_group(days_employed):
        """
        Возвращает группу стажа по значению days_employed.
        Выделяют 4 категории:
        - до 1004 дней;
        - от 1004 до 2167 дней;
        - от 2167 до 5471 дней;
        - более 5471 дней.
        """
        if days_employed < 1004:
                return 'до 1004 дней'
        elif 1004 <= days_employed < 2167:
                return 'от 1004 до 2167 дней'
        elif 2167 <= days_employed < 5471:
                return 'от 2167 до 5471 дней'
        return 'более 5471 дней'

Cоздадим отдельный столбец с категориями стажа, и в его ячейках запишем значения, возвращаемые функцией.

In [ ]:
data['days_employed_group'] = data['days_employed'].apply(days_employed_group)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,total_income_group,days_employed_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 42 лет до 53 лет,высокий доход,более 5471 дней
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 33 до 42 лет,низкий доход,от 2167 до 5471 дней
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 33 до 42 лет,средний доход,более 5471 дней
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 33 лет,высокий доход,от 2167 до 5471 дней
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,более 53 лет,средний доход,более 5471 дней
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,до 33 лет,высокий доход,до 1004 дней
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,от 42 лет до 53 лет,высокий доход,от 2167 до 5471 дней
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,от 42 лет до 53 лет,низкий доход,до 1004 дней
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,от 33 до 42 лет,малоимущие,более 5471 дней
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,от 33 до 42 лет,средний доход,от 2167 до 5471 дней


Выведем статистику по группе методом `value_counts()`.

In [ ]:
data['days_employed_group'].value_counts()

от 2167 до 5471 дней    5365
от 1004 до 2167 дней    5364
более 5471 дней         5363
до 1004 дней            5361
Name: days_employed_group, dtype: int64

#### 2.5.4 Категории для *children*.<a id="категоризация_данных__категории_для_children"></a>

Выведем статистику по количеству детей методом `value_counts()`.

In [ ]:
data['children'].value_counts()

0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Из результатов видно, что преобладаю клиенты, у которых нет детей. Поэтому предлагаю выделить 3 группы: 0 детей, 1-2 ребенка, более 3 детей.

Напишем функцию, которая принимает количество детей клиента и причисляет его к определенной категории.

In [ ]:
def children_group(children):
        """
        Возвращает группу количества детей по значению children.
        Выделяют 3 категории:
        - 0 детей;
        - 1-2 ребенка;
        - более 3 детей.
        """
        if children == 0:
                return '0 детей'
        elif (children == 1) | (children == 2):
                return '1-2 ребенка'
        return 'более 3 детей'

Cоздадим отдельный столбец с категориями количества детей, и в его ячейках запишем значения, возвращаемые функцией.

In [ ]:
data['children_group'] = data['children'].apply(children_group)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,total_income_group,days_employed_group,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 42 лет до 53 лет,высокий доход,более 5471 дней,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 33 до 42 лет,низкий доход,от 2167 до 5471 дней,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,от 33 до 42 лет,средний доход,более 5471 дней,0 детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 33 лет,высокий доход,от 2167 до 5471 дней,более 3 детей
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,более 53 лет,средний доход,более 5471 дней,0 детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,до 33 лет,высокий доход,до 1004 дней,0 детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,от 42 лет до 53 лет,высокий доход,от 2167 до 5471 дней,0 детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,от 42 лет до 53 лет,низкий доход,до 1004 дней,0 детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,от 33 до 42 лет,малоимущие,более 5471 дней,1-2 ребенка
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,от 33 до 42 лет,средний доход,от 2167 до 5471 дней,0 детей


Выведем статистику по группе методом `value_counts()`.

In [ ]:
print(data['children_group'].value_counts()) 

0 детей          14090
1-2 ребенка       6983
более 3 детей      380
Name: children_group, dtype: int64


#### 2.5.5 Категории для *purpose*.<a id="категоризация_данных__категории_для_purpose"></a>

При проведении лемматизация выяснили, что основных целей кредита 5: свадьба, недвижимость, жилье, автомобиль и образование. 

Напишем функцию, которая будет производить лемматизацию цели кредита и будет возвращать одну из пяти основных целей кредита.

In [ ]:
def key_words(row):

    purpose_lemmas_str = m.lemmatize(row)

    try:
        if 'свадьба' in purpose_lemmas_str:
            return 'свадьба'
        elif 'недвижимость' in purpose_lemmas_str:
            return 'недвижимость'
        elif 'жилье' in purpose_lemmas_str:
            return 'жилье'
        elif 'автомобиль' in purpose_lemmas_str:
            return 'автомобиль'
        elif 'образование' in purpose_lemmas_str:
             return 'образование'
        else:
            return 'другая цель'
    except:
        return None

data['purpose_categories'] = data['purpose'].apply(key_words)

--- 3.2416045665740967 seconds ---


In [ ]:
data['purpose_categories'].value_counts()

недвижимость    6350
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_categories, dtype: int64

Объединим недвижимость с жильем, так как это одна и та же цель.

In [ ]:
data['purpose_categories'] = data['purpose_categories'].replace('жилье', 'недвижимость')
data['purpose_categories'].value_counts()

недвижимость    10810
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_categories, dtype: int64

Проверим, что в столбце ***purpose_categories*** не оказалось пропущенных данных.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 17 columns):
children               21453 non-null int64
days_employed          21453 non-null int64
dob_years              21453 non-null int64
education              21453 non-null object
education_id           21453 non-null int64
family_status          21453 non-null object
family_status_id       21453 non-null int64
gender                 21453 non-null object
income_type            21453 non-null object
debt                   21453 non-null int64
total_income           21453 non-null int64
purpose                21453 non-null object
dob_years_group        21453 non-null object
total_income_group     21453 non-null object
days_employed_group    21453 non-null object
children_group         21453 non-null object
purpose_categories     21453 non-null object
dtypes: int64(7), object(10)
memory usage: 2.8+ MB


#### 2.5.6 Категории для *education*.<a id="категоризация_данных__категории_для_education"></a>

Для столбца ***education*** категоризация уже сделана, результаты хранятся в столбце ***education_id***. Проверим, что в данных нет ошибок и размерность одинаковы. Для этого воспользуемся методом `value_counts()`

In [ ]:
data['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [ ]:
data['education_id'].value_counts()

1    15172
0     5250
2      743
3      282
4        6
Name: education_id, dtype: int64

Размерности совпадают. С данными все хорошо.

#### 2.5.7 Категории для *family_status*.<a id="категоризация_данных__категории_для_family_status"></a>

Для столбца ***family_status*** категоризация уже сделана, результаты хранятся в столбце ***family_status_id***. Проверим, что в данных нет ошибок и размерность одинаковы. Для этого воспользуемся методом `value_counts()`

In [ ]:
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4150
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [ ]:
data['family_status_id'].value_counts()

0    12339
1     4150
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

Размерности совпадают. С данными все хорошо.

### Вывод

В результате категоризации данных были выделены и добавлены в таблицу следующие группы:
- в зависимости от возраста клиента (столбец ***dob_years_group***);
- в зависимости от уровня дохода клиента (столбец ***total_income_group***);
- в зависимости от трудового стажа клиента (столбец ***days_employed_group***);
- в зависимости от количества детей у клиента (столбец ***children_group***);
- в зависимости от цели получения кредита (столбец ***purpose_categories***).

Эти категории помогут нам ответить на поставленные вопросы заказчиком.

### Шаг 3. Ответы на вопросы<a id="ответы_на_вопросы"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для этого сгруппируем данные по столбцу ***children_group*** относительно столбца ***debt***, используя метод `groupby()`. Посчитаем количество строк и суммарное значение по строкам, используя метод `agg()`

In [ ]:
data_children_grouped = data.groupby('children_group').agg({'debt':['sum', 'count']})
data_children_grouped['share'] = data_children_grouped['debt']['sum'] / data_children_grouped['debt']['count']
data_children_grouped.sort_values(by = 'share', ascending = False)

debt            share
                 sum  count          
children_group                       
1-2 ребенка      647   6983  0.092654
более 3 детей     31    380  0.081579
0 детей         1063  14090  0.075444

### Вывод

Из результатов видно, что клиенты у которых есть дети, чаще имеют проблемы с возвратом кредита в срок. Клиенты, у которых 1-2 ребенка, сильнее подвержены к просрочкам по оплате кредита, нежели, чем клиенты, у которых 3 и более детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для этого сгруппируем данные по столбцу ***family_status*** относительно столбца ***debt***, используя метод `groupby()`. Посчитаем количество строк и суммарное значение по строкам, используя метод `agg()`

In [ ]:
data_family_status_grouped = data.groupby('family_status').agg({'debt':['sum', 'count']})
data_family_status_grouped['share'] = data_family_status_grouped['debt']['sum'] / data_family_status_grouped['debt']['count']
data_family_status_grouped.sort_values(by = 'share', ascending = False)

debt            share
                       sum  count          
family_status                              
не женат / не замужем  274   2810  0.097509
гражданский брак       388   4150  0.093494
женат / замужем        931  12339  0.075452
в разводе               85   1195  0.071130
вдовец / вдова          63    959  0.065693

### Вывод

Клиенты, которые никогда не были в официально узаконенных отношениях, более предрасположены к просрочкам, чем клиенты, которые состоят или состояли когда-либо в узаконенных отношениях.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для этого создадим сводную таблицу, используя метод `pivot_table()`.

In [ ]:
data_pivot_total_income = data.pivot_table(index='total_income_group', values='debt', aggfunc=['sum', 'count'])
data_pivot_total_income['share'] = data_pivot_total_income['sum'] / data_pivot_total_income['count']
data_pivot_total_income.sort_values(by = 'share', ascending = False)

,sum,count,share
,debt,debt,
total_income_group,,,
низкий доход,428,4887,0.087579
средний доход,552,6375,0.086588
малоимущие,384,4886,0.078592
высокий доход,377,5305,0.071065


### Вывод

Результаты весьма интересны. Ожидаемо, что клиенты с высоким доходом менее всего предрасположены к просрочкам по выплате кредита. Но люди с низким и средним уровнем дохода оказались более предрасположены к просрочкам по выплате кредита, нежели клиенты с низким уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

Для этого создадим сводную таблицу, используя метод `pivot_table()`.

In [ ]:
data_pivot_purpose = data.pivot_table(index='purpose_categories', values='debt', aggfunc=['sum', 'count'])
data_pivot_purpose['share'] = data_pivot_purpose['sum'] / data_pivot_purpose['count']
data_pivot_purpose.sort_values(by = 'share', ascending = False)

,sum,count,share
,debt,debt,
purpose_categories,,,
автомобиль,403,4306,0.093590
образование,370,4013,0.092200
свадьба,186,2324,0.080034
недвижимость,782,10810,0.072340


### Вывод

Клиенты, берущие кредит на автомобиль или образование, чаще имеют проблемы с возвратом кредита в срок, чем клиенты, берущие кредит для покупки недвижимости или проведения свадьбы.

### Шаг 4. Общий вывод<a id="общий_вывод"></a>

В результате проделанной работы нам удалось:
1. Ознакомиться с данными, содержащимися в файле data.csv;
2. Найти и обработать аномальные данные, выбросы, пропущенные значения в каждом столбце. Основные проблемы были в следующих столбцах:
    - Столбец children. Были аномальные данные со значениями -1, 20. Было предположение, что при заполнении данных были допущены опечатки и случайно добавлен знак минус для значения -1 и лишний ноль для значения 20. Поэтому заменили -1 на 1, а 20 на 2;                                                                                   
    ***Рекомендации***: Добавить проверку ввода в анкете отрицательного значения и анамально большого (допустим более 9) значения для столбца *children*.
       
    - Столбец *dob_years*. Были аномальные данные со значениями 0 (у 101 клиента возраст был указан 0). Было решение заменить нулевые значения на средний возраст в зависимости от типа занятости клиента;                                                     
    ***Рекомендации***: Добавить проверку ввода в анкете отрицательного значения, нулевого значения и анамально большого (допустим более 100) значения для столбца *dob_years*.
    
    - Столбец *education*. В столбце образование знаечния были написаны в разном регистре. Было решение привести все значения в этом столбце к нижнему регистру;                                             
    ***Рекомендации***: в анкете автоматически приводить написанное значение к нижнему регистру.
    
    - Столбец *gender*. Был найден клиент со значением XNA. Так как определить пол клиента не получилось возможным, было решение удалить эту строчку из таблицы. Такой клиент всего-лишь один, поэтому на результатах исследования это не сказалось;    
    ***Рекомендации***: в анкете для столбца *gender* сделать раскрывающийся список со значениями 'F' или 'M'.
    
    - Столбец *total_income*. В столбце было найдено 2174 пропущенных значений. Было решение заменить их на знаечние медианы зарплаты в зависимости от типа занятости;   
    ***Рекомендации***: не принимать анкету, если знаечние *total_income*  пустое.
    
    - Столбец *days_employed*. В столбце было найдено 2174 пропущенных значений. Было решение заменить их на знаечние медианы зарплаты в зависимости от типа занятости. Также в столбце были найдены выбросы (все выбросы пренадлежат пенсионерам и безработным). Было предположение, что для этих клиентов стаж указан в часах, а не в днях (например, данные объеденены из нескольких таблиц), поэтому для этих клиентов значения были переведены в дни. А вобще я стаж указывал бы в годах. а не днях;  
     ***Рекомендации***: не принимать анкету, если знаечние *days_employed*  больше, чем (age - 18) * 365.   
     
3. Найти и удалить дубликаты. Был найден 71 дубликат. Все эти строчки были удалены. Возможная причина возникновения дубликатов - это человеческий фактор, когда сотрудник банка случайно отправляет несколько заявок на кредит. Еще возможная причина, что клиент сам несколько раз подавал заявку на кредит, но тогда значения трудового стажа должны были быть разными.
4. Поменять тип данных с float на int для столбцов *total_income* и *days_employed*, так как уровень зп и стажа должны быть целыми числами;
5. Провели лемматизацию целей кредита и выделить основные цели. Их оказалось 5: недвижимость, жилье, свадьба, автомобиль, образование;
6. Провести категоризацию данных. Выделили следующие категории: 
    - Для столбца *dob_years*: до 33 лет, от 33 до 42 лет, от 42 лет до 53 лет, более 53 лет;
    - Для столбца *total_income*: до 103460, от 103460 до 142594, от 142594 до 196593, более 196593;
    - Для столбца *days_employed*: до 1004 дней, от 1004 до 2167 дней, от 2167 до 5471 дней, более 5471 дней;
    - Для столбца *children*: 0 детей, 1-2 ребенка, более 3 детей;
    - Для столбца *purpose*: недвижимость, жилье, свадьба, автомобиль, образование, другая цель.
6. Ответить на поставленные вопросы, аргументируя свои ответы результатами исследования;

Перед выполнением анализа перед нами стояли следующие вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

В результате проделанного анализа удалось выяснить:
   1. Количество детей влияет на возврат кредита в срок. Категории расположены в возрастающей вероятности задолженности по кредиту: 0 детей -> 3 и более детей -> 1-2 ребенка;
   2. Семейное положение влияет на возврат кредита в срок. Категории расположены в возрастающей вероятности задолженности по кредиту: вдовец / вдова -> в разводе -> женат / замужем -> гражданский брак -> не женат / не замужем;
   3. Уровень дохода влияет на возврат кредита в срок. Категории расположены в возрастающей вероятности задолженности по кредиту: высокий доход -> малоимущие -> средний доход -> низкий доход;
   4. Цели кредита также влияют на возврат кредита в срок. Категории расположены в возрастающей вероятности задолженности по кредиту: недвижимость -> свадьба -> образование -> автомобиль.
    
Если у клиента 0 детей, он вдовец / вдова, у него высокий доход и цель кредита недвижимость, то вероятность его возврата кредита в срок наивысшая.
Если же у клиента 1-2 ребенка, он не женат / не замужем, у него низкий доход и цель кредита автомобиль, то вероятность его возврата кредита в срок наименьшая.